In [ ]:
# 扩展 Java 内存
options(java.parameters = "-Xmx8g")

# 加载 C4R 核心库
library(loadeR)
library(loadeR.2nc)
library(transformeR)
library(downscaleR)
library(downscaleR.keras)
library(visualizeR)
library(climate4R.value)

# 其他有用的库
library(magrittr)  # 用于管道操作 %>%
library(RColorBrewer)
library(gridExtra)
library(ggplot2)

# 创建数据、模型和图形文件夹
if (!dir.exists("./data/pr")) dir.create("./data/pr", recursive = TRUE, showWarnings = FALSE)
if (!dir.exists("./data/tas")) dir.create("./data/tas", showWarnings = FALSE)
if (!dir.exists("./models/pr")) dir.create("./models/pr", recursive = TRUE, showWarnings = FALSE)
if (!dir.exists("./models/tas")) dir.create("./models/tas", showWarnings = FALSE)
if (!dir.exists("./figures")) dir.create("./figures", showWarnings = FALSE)

In [ ]:
# --- 在 'pr' 或 'tas' 之间选择一个作为预测目标 ---
predictand <- 'pr'
# ----------------------------------------------

# 定义是否将数据文件压缩为 .gz 格式
compress <- TRUE
extension <- if (isTRUE(compress)) {
   "rds.gz"
} else {
  "rds"
}

In [ ]:
# 1. 定义 predictor 变量
vars  <- c(
  "z@500","z@700","z@850",       # 不同气压层上的位势高度
  "hus@500","hus@700","hus@850", # 不同气压层上的比湿度
  "ta@500","ta@700","ta@850",    # 不同气压层上的气温
  "ua@500","ua@700","ua@850",    # 不同气压层上的纬向风
  "va@500","va@700","va@850"     # 不同气压层上的经向风
)

In [ ]:
x <- readRDS(sprintf("./data/x_ERA-Interim.%s", extension))
y <- readRDS(sprintf("./data/%s/y.%s", predictand, extension))

In [ ]:
# 1. 定义 CNN 模型结构
# 降水 (pr) 和 气温 (tas) 的模型结构不同
modelCNN <- function(inp) {
  padding = switch(predictand, pr = "same", tas = "valid")
  filters.l1 = 50
  filters.l2 = 25
  filters.l3 = switch(predictand, pr = 1, tas = 10)
  # 输入层
  inputs <- layer_input(shape = dim(inp$x.global)[2:4])
  # 隐藏层
  l1 = layer_conv_2d(inputs, filters = filters.l1, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l2 = layer_conv_2d(    l1, filters = filters.l2, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l3 = layer_conv_2d(    l2, filters = filters.l3, kernel_size = c(3,3), activation = 'relu', padding = padding)
  l4 = layer_flatten(    l3)
  # 输出层 (根据预测目标而变化)
  if (predictand == "pr") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'sigmoid')
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' )
    l53 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear' )
    outputs <- layer_concatenate(list(l51,l52,l53))
  } else if (predictand == "tas") {
    l51 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear')
    l52 = layer_dense(l4, units = dim(inp$y$Data)[2], activation = 'linear')
    outputs <- layer_concatenate(list(l51, l52))
  }
  model <- keras_model(inputs = inputs, outputs = outputs)
}



In [ ]:
# 2. 标准化 predictor 和处理 predictand
x <- scaleGrid(x,type = "standardize")
if (predictand == "pr") {
  # 对于降水，将小于1mm的日降水视为无雨日
  y <- binaryGrid(
    gridArithmetics(y, 0.99, operator = "-"),
    condition = "GE",
    threshold = 0,
    partial = TRUE
  )
}

In [ ]:
# 3. 准备用于 Keras 训练的数据格式
xy.train <- prepareData.keras(
  x = x,
  y = y,
  first.connection = "conv",
  last.connection = "dense",
  channels = "last"
)

In [ ]:
# 4. 训练模型
# 警告：运行此单元格大约需要1小时
cnnmodel.name <- switch(predictand,
  pr="CNN1",
  tas="CNN10"
)

In [ ]:
cnnloss <- switch(predictand,
  pr = bernouilliGammaLoss(last.connection = "dense"),
  tas = gaussianLoss(last.connection = "dense")
)

In [ ]:
output.file <- sprintf('./models/%s/%s.h5', predictand, cnnmodel.name)
if (! file.exists(output.file)) {

  # 训练CNN模型
  downscaleTrain.keras(
    obj = xy.train,
    model = modelCNN(xy.train),
    clear.session = TRUE,
    compile.args =
      list(
        "loss" = cnnloss,
        "optimizer" = optimizer_adam(lr = 0.0001)
      ),
    fit.args =
      list(
        "batch_size" = 5,
        "epochs" = 5,
        "validation_split" = 0.1,
        "verbose" = 1,
        "callbacks" = list(
           callback_early_stopping(patience = 30),
           callback_model_checkpoint(
             filepath = output.file,
             monitor = 'val_loss',
             save_best_only = TRUE
           )
        )
     )
  )
}

In [ ]:
if (predictand == "pr"){
  xy.test <- prepareNewData.keras(x, xy.train)
  pred_ocu_train <- cnn.predict(xy.test) %>% subsetGrid(var = "p")
}